<a href="https://colab.research.google.com/github/EdwardFang09/IEE4912/blob/main/Tugas_akhir_(documentation_v2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Sora assistant

Features:
- Google search image
- Mentimeter
- Kahoot
- Start class and end class
- Web Based CRUD

Faster whisper
https://github.com/SYSTRAN/faster-whisper

In [ ]:
import torch
print(torch.__version__)
torch.cuda.is_available()

2.6.0+cu126


True

Without trained model

In [ ]:
import speech_recognition as sr
import whisper
import warnings
import time
import webbrowser
import pyttsx3
import os
from faster_whisper import WhisperModel
import sqlite3
import datetime
from thefuzz import fuzz  # Import thefuzz
import numpy as np

# Suppress warnings
warnings.filterwarnings("ignore")

# Initialize speech recognition, TTS engine, and Whisper model
recognizer = sr.Recognizer()
engine = pyttsx3.init()

model_size = "turbo"  # or smaller, depending on your resources
base_model = WhisperModel(model_size, device="cuda", compute_type="int8_float16") # or cpu

assistant_name = "sora mk.0"
should_run = True
source = sr.Microphone()

# Database setup (SQLite)
DATABASE_FILE = "class_data.db"  # Name of your database file
conn = sqlite3.connect(DATABASE_FILE)
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS classes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        start_time TEXT,
        end_time TEXT
    )
''')
conn.commit()


def respond(text):
    engine.say(text)
    engine.runAndWait()

def listen_for_command():
    with source as s:
        print("Listening for commands...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        with open("command.wav", "wb") as f:
            f.write(audio.get_wav_data())

        command, info = base_model.transcribe(r"command.wav", language='en')
        print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

        if command:
            full_command_text = ""
            for segment in command:
                print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
                full_command_text += segment.text + " "
            full_command_text = full_command_text.strip().lower()

            # Check for "sora" first
            if "sora" in full_command_text:
                print("Sora detected. Proceeding with command recognition.")
                # Proceed to command matching
            else:
                print("No 'sora' detected. Waiting for trigger word.")
                return None  # Wait for the trigger word

            # Fuzzy matching logic (now only runs IF "sora" is present)
            best_match = None
            best_score = 0

            commands = {
                "find": "find",
                "open mentimeter": "open mentimeter",
                "open kahoot": "open kahoot",
                "start class": "start class",
                "end class": "end class",
                "exit": "exit",
                "google": "google",
                "chrome":  "chrome",
                "browser": "browser"

            }

            scores = []
            for cmd, target in commands.items():
                score = fuzz.ratio(full_command_text, target)
                scores.append(score)

                if score > best_score:
                    best_score = score
                    best_match = cmd

            # One Sigma Calculation and Threshold
            scores_array = np.array(scores)
            average_score = np.mean(scores_array)
            standard_deviation = np.std(scores_array)

            confidence_threshold = average_score + standard_deviation

            print(f"Average Score: {average_score}")
            print(f"Standard Deviation: {standard_deviation}")
            print(f"Confidence Threshold: {confidence_threshold}")


            if best_score >= confidence_threshold:
                print("Matched command:", best_match)
                return best_match
            else:
                print("Command not recognized with sufficient confidence.")
                return None

        return None

    except sr.UnknownValueError:
        print("Could not understand audio. Please try again.")
        return None
    except sr.RequestError:
        print("Unable to access the speech recognition API.")
        return None



def perform_command(command):
    global should_run
    if command:
        print("Command: ", command)
        if "find" in command:
            query = command.replace("find", "").strip()
            respond(f"Finding images of {query}")
            try:
                search_url = f"https://www.google.com/search?q={query}&tbm=isch"
                webbrowser.open(search_url)
            except Exception as e:
                respond(f"Sorry, I encountered an error during image search: {e}")
        elif "open mentimeter" in command:
            respond("Opening Mentimeter.")
            webbrowser.open("https://www.mentimeter.com/")
        elif "open kahoot" in command:
            respond("Opening Kahoot.")
            webbrowser.open("https://kahoot.com/")
        elif "start class" in command:
            start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute("INSERT INTO classes (start_time) VALUES (?)", (start_time,))  # Only start time
            conn.commit()
            respond("Class started.")
        elif "end class" in command:
            end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute("UPDATE classes SET end_time =? WHERE end_time IS NULL ORDER BY id DESC LIMIT 1", (end_time,)) # Update the latest started class
            conn.commit()
            respond("Class ended.")
        elif "google" in command or "chrome" in command or "browser" in command:
            respond("Opening Google.")
            webbrowser.open("www.google.com")  # Or "https://www.google.com"
        elif "exit" in command:
            should_run = False
        else:
            respond("I don't understand that command.")


def main():
    while should_run:
        command = listen_for_command()  # Directly listen for command
        perform_command(command)
        time.sleep(1)
    respond("Goodbye.")
    conn.close() # Close database connection when exiting

if __name__ == "__main__":
    main()

Listening for commands...
Detected language 'en' with probability 1.000000
[0.00s -> 2.36s]  Okay, so I'll open Google.
No 'sora' detected. Waiting for trigger word.
Listening for commands...
Detected language 'en' with probability 1.000000
[0.00s -> 2.00s]  Hey Sora Open Google
Sora detected. Proceeding with command recognition.
Average Score: 27.555555555555557
Standard Deviation: 12.893677271447697
Confidence Threshold: 40.44923282700326
Matched command: google
Command:  google
Listening for commands...
Detected language 'en' with probability 1.000000
[0.00s -> 2.20s]  Sora, buka Google.
Sora detected. Proceeding with command recognition.
Average Score: 23.333333333333332
Standard Deviation: 14.514360704718161
Confidence Threshold: 37.84769403805149
Matched command: google
Command:  google
Listening for commands...
Detected language 'en' with probability 1.000000
[0.00s -> 2.04s]  Surah, buka mentimeter.
No 'sora' detected. Waiting for trigger word.
Listening for commands...
Detecte

KeyboardInterrupt: 

Trained model

In [ ]:
import speech_recognition as sr
import warnings
import time
import webbrowser
import pyttsx3
import os
from faster_whisper import WhisperModel
import sqlite3
import datetime
from thefuzz import fuzz
import numpy as np

# Suppress warnings
warnings.filterwarnings("ignore")

# --- Configuration ---

# --- Cache Directory ---
# Use a simple, absolute path to avoid issues with relative paths.
# Ensure this directory exists OR the script has permission to create it.
# If you prefer the script's directory, change back to "./models_cache",
# but absolute paths are often more reliable for caching.
# On Linux/Jetson, this might be "/home/user/hf_models_cache" or "/mnt/external/hf_cache"
CUSTOM_CACHE_DIR = "./models" # Example for Windows - CHANGE IF NEEDED

# --- Model Configuration ---
MODEL_PATH = "EdwardFang09/whisper-base-TA-2025_v2-ct2" # Use the standard OpenAI base model
DEVICE = "cuda" # Ensure CUDA is correctly set up

# --- Compute Type ---
# Start with "default" or "float32" to ensure model downloads correctly first.
# If successful, you can change back to "float16" for potential speed/memory benefits.
# "float16" might require specific GPU capabilities or CUDA versions.
COMPUTE_TYPE = "default"
# COMPUTE_TYPE = "float16" # Change back later if "default" works

ASSISTANT_NAME = "sora" # Trigger word (lowercase)
DATABASE_FILE = "class_data.db"
AUDIO_FILENAME = "command_temp.wav" # Temporary audio file

# --- Command Recognition Settings ---
CONFIDENCE_THRESHOLD_TYPE = "dynamic" # 'dynamic' or 'fixed'
FIXED_CONFIDENCE_THRESHOLD = 75 # Used if CONFIDENCE_THRESHOLD_TYPE is 'fixed'
# Minimum threshold if using dynamic, prevents overly low thresholds
DYNAMIC_MIN_THRESHOLD = 60

# Define commands: Keys are for internal use, Values are what fuzzy matching compares against
COMMANDS = {
    "search": "search for", # User says "sora search for cats"
    "open_menti": "open mentimeter",
    "open_kahoot": "open kahoot",
    "start_class": "start class",
    "end_class": "end class",
    "exit": "exit",
    "google": "open google", # Changed for clarity
    # Add more commands here
}

# --- Initialization ---
print("Initializing...")
recognizer = sr.Recognizer()
engine = pyttsx3.init()

# --- Setup Cache Directory ---
print(f"\n--- Cache Directory Setup ---")
try:
    # Ensure the chosen cache directory exists
    os.makedirs(CUSTOM_CACHE_DIR, exist_ok=True)
    abs_cache_path = os.path.abspath(CUSTOM_CACHE_DIR)
    print(f"Ensured cache directory exists: {abs_cache_path}")

    # Test write permission (optional but helpful)
    test_file_path = os.path.join(CUSTOM_CACHE_DIR, 'permission_test.txt')
    with open(test_file_path, 'w') as f:
        f.write('test')
    os.remove(test_file_path)
    print(f"Write permissions confirmed for: {abs_cache_path}")
except Exception as e:
    abs_cache_path = os.path.abspath(CUSTOM_CACHE_DIR)
    print(f"\n--- ERROR: Cache Directory Problem ---")
    print(f"Could not create or write to cache directory '{abs_cache_path}': {e}")
    print("Please ensure the path is correct and you have write permissions.")
    print("Script will now exit.")
    exit() # Exit if cache directory setup fails
print(f"--- Cache Setup OK ---")

# --- Load Model (Only Once) ---
print(f"\n--- Model Loading ---")
base_model = None # Initialize base_model to None
try:
    print(f"Attempting to load model: {MODEL_PATH}")
    print(f"Using cache directory: {abs_cache_path}")
    print(f"Device: {DEVICE}, Compute Type: {COMPUTE_TYPE}")
    print("Calling WhisperModel constructor...") # Log before call

    # *** This is the ONLY place the model is loaded ***
    base_model = WhisperModel(
        MODEL_PATH,
        device=DEVICE,
        compute_type=COMPUTE_TYPE,
        download_root=CUSTOM_CACHE_DIR, # Use the verified cache path
        local_files_only=False # IMPORTANT: Allow downloading
    )

    print("WhisperModel constructor finished.") # Log after call

    # Verify model object was created
    if base_model:
         print("Model loaded successfully into 'base_model' variable.")
         print(f"--- Model Loading OK ---")
    else:
         # This case shouldn't happen if WhisperModel succeeded without error, but check anyway
         print("WARNING: WhisperModel call finished but 'base_model' is still None.")
         raise RuntimeError("Model object not created after WhisperModel call.")

except Exception as e:
    print(f"\n--- FATAL ERROR: Model Loading Failed ---")
    print(f"Error details: {e}")
    print("\nTroubleshooting suggestions:")
    print(f"1. Manually delete the cache directory and try again: {abs_cache_path}")
    print(f"2. Ensure you have a stable internet connection (for first download).")
    print(f"3. Verify model ID '{MODEL_PATH}' exists on Hugging Face Hub.")
    print(f"4. Check you have enough disk space in: {abs_cache_path}")
    print(f"5. Ensure selected device ('{DEVICE}') is available and drivers are installed.")
    print(f"6. Check compute type ('{COMPUTE_TYPE}') compatibility. Try 'default' or 'float32' if issues persist.")
    print(f"7. Check if Anti-Virus/Firewall software is interfering with downloads or file access.")
    print(f"8. Ensure the cache path '{abs_cache_path}' is simple and accessible.")
    print("\nScript will now exit.")
    # *** IMPORTANT: Ensure script exits if model loading fails ***
    exit()

# --- Database Setup ---
print(f"\n--- Database Setup ---")
try:
    print(f"Connecting to database: {DATABASE_FILE}")
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS classes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            start_time TEXT,
            end_time TEXT
        )
    ''')
    conn.commit()
    print("Database initialized.")
    print(f"--- Database Setup OK ---")
except Exception as e:
    print(f"--- FATAL ERROR: Database Setup Failed ---")
    print(f"Error details: {e}")
    print("\nScript will now exit.")
    exit()

# --- Core Functions ---
def respond(text):
    """Sends text to the TTS engine."""
    print(f"Responding: {text}")
    try:
        engine.say(text)
        engine.runAndWait()
    except Exception as e:
        print(f"Error during TTS processing: {e}")

def listen_for_command(microphone_source):
    """Listens for audio, transcribes, checks trigger word, and matches command."""
    # Ensure base_model is loaded before proceeding
    if not base_model:
         print("ERROR: Model was not loaded. Cannot transcribe.")
         return None

    with microphone_source as s:
        print("\nListening...")
        try:
            recognizer.adjust_for_ambient_noise(s, duration=0.5)
            # Listen with timeouts to prevent indefinite blocking
            audio = recognizer.listen(s, timeout=7, phrase_time_limit=15)
        except sr.WaitTimeoutError:
            print("No speech detected within timeout.")
            return None
        except Exception as e:
            print(f"Error during listening phase: {e}")
            return None

    try:
        # Save audio temporarily for faster-whisper
        with open(AUDIO_FILENAME, "wb") as f:
            f.write(audio.get_wav_data())

        # Transcribe using faster-whisper
        # Consider adding VAD filter if background noise is an issue: vad_filter=True
        segments, info = base_model.transcribe(AUDIO_FILENAME, language='en', beam_size=5)
        print(f"Detected language '{info.language}' with probability {info.language_probability:.2f}")

        full_command_text = "".join(segment.text for segment in segments).strip().lower()
        print(f"Transcribed Text: '{full_command_text}'")

        # --- Trigger Word Check ---
        if not full_command_text or not full_command_text.startswith(ASSISTANT_NAME):
             print(f"No command or trigger word '{ASSISTANT_NAME}' not detected at start.")
             return None

        # Extract command phrase after trigger word
        command_phrase = full_command_text.replace(ASSISTANT_NAME, "", 1).strip()
        if not command_phrase:
            print("Trigger word heard, but no command followed.")
            # Maybe respond("Yes?") or similar? For now, just ignore.
            return None
        print(f"Command Phrase (after trigger): '{command_phrase}'")

        # --- Fuzzy Matching Logic ---
        best_match_key = None
        best_score = 0
        scores = []

        print("Comparing against commands:")
        for cmd_key, target_phrase in COMMANDS.items():
            # Use partial_ratio for commands expecting follow-up words (like 'search for')
            # Use regular ratio for commands needing exact matches
            if cmd_key == "search":
                 score = fuzz.partial_ratio(command_phrase, target_phrase)
            else:
                 score = fuzz.ratio(command_phrase, target_phrase)

            print(f"- vs '{target_phrase}': Score {score}")
            scores.append(score)

            if score > best_score:
                best_score = score
                best_match_key = cmd_key

        # --- Confidence Check ---
        if CONFIDENCE_THRESHOLD_TYPE == "dynamic":
            if not scores: # Avoid numpy errors if COMMANDS is empty
                 confidence_threshold = DYNAMIC_MIN_THRESHOLD
            else:
                 scores_array = np.array(scores)
                 average_score = np.mean(scores_array)
                 standard_deviation = np.std(scores_array)
                 # Calculate dynamic threshold but ensure it's not below the minimum
                 confidence_threshold = max(average_score + standard_deviation, DYNAMIC_MIN_THRESHOLD)
            print(f"Dynamic Threshold Calculated: {confidence_threshold:.2f}")
        else: # fixed threshold
            confidence_threshold = FIXED_CONFIDENCE_THRESHOLD
            print(f"Using Fixed Threshold: {confidence_threshold}")

        # Check if the best match meets the threshold
        if best_match_key and best_score >= confidence_threshold:
            print(f"Confident Match: Key='{best_match_key}', Score={best_score}, Threshold={confidence_threshold:.2f}")
            # Return the matched key AND the specific phrase spoken after the trigger word
            return (best_match_key, command_phrase)
        else:
            print(f"Command not recognized with sufficient confidence (Best Score: {best_score} < Threshold: {confidence_threshold:.2f}).")
            # Optional: Respond that the command wasn't understood
            # respond("Sorry, I didn't quite catch that command.")
            return None

    except sr.UnknownValueError:
        print("Could not understand audio clearly after listening.")
        return None
    except sr.RequestError as e:
        print(f"Speech recognition service error (if using online recognizer): {e}")
        return None
    except Exception as e:
        print(f"Error during transcription or command matching: {e}")
        # Log the traceback for debugging if needed:
        # import traceback
        # traceback.print_exc()
        return None
    finally:
        # Clean up temporary audio file
        if os.path.exists(AUDIO_FILENAME):
             try:
                 os.remove(AUDIO_FILENAME)
             except OSError as e:
                 print(f"Warning: Could not delete temp audio file {AUDIO_FILENAME}: {e}")


def perform_command(command_data):
    """Executes the action based on the matched command key and original phrase."""
    global should_run # Need global scope to modify should_run for exit command
    if not command_data:
        return # No valid command was recognized

    command_key, original_phrase = command_data # Unpack the tuple
    print(f"\n--- Executing Command ---")
    print(f"Key: '{command_key}', Original Phrase: '{original_phrase}'")

    # Use if/elif based on the command_key returned by fuzzy matching
    if command_key == "search":
        # Extract the query part after "search for"
        # Assumes the target phrase in COMMANDS is "search for"
        query = original_phrase.replace(COMMANDS[command_key], "").strip()
        if query:
            respond(f"Searching Google Images for {query}")
            try:
                search_url = f"https://www.google.com/search?q={query}&tbm=isch"
                webbrowser.open(search_url)
            except Exception as e:
                respond(f"Sorry, I encountered an error opening the browser for image search: {e}")
        else:
            respond("What would you like me to search for? Please say 'search for' followed by your query.")

    elif command_key == "open_menti":
        respond("Opening Mentimeter.")
        try:
            webbrowser.open("https://www.mentimeter.com/")
        except Exception as e:
             respond(f"Sorry, I encountered an error opening Mentimeter: {e}")

    elif command_key == "open_kahoot":
        respond("Opening Kahoot.")
        try:
            webbrowser.open("https://kahoot.com/")
        except Exception as e:
             respond(f"Sorry, I encountered an error opening Kahoot: {e}")

    elif command_key == "start_class":
        try:
            start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute("INSERT INTO classes (start_time) VALUES (?)", (start_time,))
            conn.commit()
            respond("Class started and time logged.")
        except Exception as e:
            respond(f"Sorry, failed to log class start time: {e}")

    elif command_key == "end_class":
        try:
            # Find the ID of the latest class entry that hasn't ended yet
            cursor.execute("SELECT id FROM classes WHERE end_time IS NULL ORDER BY id DESC LIMIT 1")
            last_class = cursor.fetchone()
            if last_class:
                end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                cursor.execute("UPDATE classes SET end_time = ? WHERE id = ?", (end_time, last_class[0]))
                conn.commit()
                respond("Class ended and time logged.")
            else:
                respond("No active class found in the log to end.")
        except Exception as e:
            respond(f"Sorry, failed to log class end time: {e}")

    elif command_key == "google":
        respond("Opening Google.")
        try:
            webbrowser.open("https://www.google.com")
        except Exception as e:
             respond(f"Sorry, I encountered an error opening Google: {e}")

    elif command_key == "exit":
        respond("Exiting program now.")
        should_run = False # Set flag to stop the main loop

    else:
        # This case should ideally not be reached if COMMANDS dict is correct
        respond(f"I understood the command key '{command_key}' but don't have an action defined for it.")

    print(f"--- Command Execution Finished ---")


# --- Main Application Loop ---
def main():
    global should_run # Allow modifying the global flag
    should_run = True # Ensure the loop runs at least once

    print("\n" + "=" * 40)
    print("      VOICE ASSISTANT INITIALIZED      ")
    print("=" * 40)
    respond(f"Assistant {ASSISTANT_NAME} activated and ready.")
    print(f"\nSay '{ASSISTANT_NAME}' followed by a command.")
    print(f"Examples:")
    for key, phrase in COMMANDS.items():
        if key == "search":
             print(f"- '{ASSISTANT_NAME} {phrase} cats'")
        else:
             print(f"- '{ASSISTANT_NAME} {phrase}'")
    print("-" * 40)

    # Initialize microphone object once
    try:
         microphone = sr.Microphone()
    except Exception as e:
         print(f"\n--- FATAL ERROR: Microphone Setup Failed ---")
         print(f"Could not initialize microphone: {e}")
         print("Please ensure a microphone is connected and recognized by the system.")
         print("Check microphone permissions if necessary.")
         print("Script will now exit.")
         exit()

    # Main loop: Listen -> Transcribe -> Match -> Execute
    while should_run:
        command_data = listen_for_command(microphone) # Pass the microphone object
        if should_run: # Check if exit command was processed in perform_command
             perform_command(command_data)
        # Brief pause can sometimes help, but listening timeout mostly handles waiting
        # time.sleep(0.1)

    # --- Cleanup ---
    print("\n--- Shutting Down ---")
    respond("Goodbye.")
    if conn:
        try:
             conn.close()
             print("Database connection closed.")
        except Exception as e:
             print(f"Error closing database connection: {e}")
    print("Program finished.")

# --- Entry Point ---
if __name__ == "__main__":
    main()

Initializing...

--- Cache Directory Setup ---
Ensured cache directory exists: c:\Users\edwar\Downloads\models
Write permissions confirmed for: c:\Users\edwar\Downloads\models
--- Cache Setup OK ---

--- Model Loading ---
Attempting to load model: EdwardFang09/whisper-base-TA-2025_v2-ct2
Using cache directory: c:\Users\edwar\Downloads\models
Device: cuda, Compute Type: default
Calling WhisperModel constructor...


model.bin:   0%|          | 0.00/145M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

WhisperModel constructor finished.
Model loaded successfully into 'base_model' variable.
--- Model Loading OK ---

--- Database Setup ---
Connecting to database: class_data.db
Database initialized.
--- Database Setup OK ---

      VOICE ASSISTANT INITIALIZED      
Responding: Assistant sora activated and ready.

Say 'sora' followed by a command.
Examples:
- 'sora search for cats'
- 'sora open mentimeter'
- 'sora open kahoot'
- 'sora start class'
- 'sora end class'
- 'sora exit'
- 'sora open google'
----------------------------------------

Listening...
No speech detected within timeout.

Listening...


KeyboardInterrupt: 

In [ ]:
pip install -U ctranslate2 transformers huggingface_hub